# Examples for the AbsComopnent Class

In [3]:
# import
from linetools.isgm import abscomponent as lt_abscomp

## Instantiate

In [10]:
reload(lt_abscomp)
abscomp = lt_abscomp.AbsComponent((10.0*u.deg, 45*u.deg), (14,2), 1.0, [-300,300]*u.km/u.s)
abscomp

[AbsComponent: 00:40:00 +45:00:00, Zion=(14,2), z=1]